In [1]:
import numpy as np
import pandas as pd
import random as rand
import math
import matplotlib.pyplot as plt
from scipy.stats import norm
from sys import maxsize

In [2]:
a=np.loadtxt(open("P3b_train_data.csv", "rb"))
xs=a[0:100,0]
labels=a[0:100,1]
data = {'x': xs, 'label': labels}
df = pd.DataFrame(data=data)

In [3]:
# ### Setup
# # set random seed
# rand.seed(42)

# # 2 clusters
# # not that both covariance matrices are diagonal
# mu1 = [0, 5]
# sig1 = [ [2, 0], [0, 3] ]

# mu2 = [5, 0]
# sig2 = [ [4, 0], [0, 1] ]

In [4]:
# # generate samples
# x1, y1 = np.random.multivariate_normal(mu1, sig1, 100).T
# x2, y2 = np.random.multivariate_normal(mu2, sig2, 100).T

# xs = np.concatenate((x1, x2))
# ys = np.concatenate((y1, y2))
# labels = ([1] * 100) + ([2] * 100)

# data = {'x': xs, 'y': ys, 'label': labels}
# df = pd.DataFrame(data=data)

In [5]:
# # inspect the data
# df.head()
# df.tail()

# fig = plt.figure()
# plt.scatter(data['x'], data['y'], 24, c=data['label'])
# fig.savefig("true-values.png")

In [6]:
### Expectation-maximization

# initial guesses - intentionally bad
guess = { 'mu1': [-1],
          'sig1': [2] ,
          'mu2': [5],
          'sig2': [2],
          'lambda1': [0.6],
          'lambda2': [0.4]
        }

In [7]:
# probability that a point came from a Guassian with given parameters
# note that the covariance must be diagonal for this to work
def prob(val, mu, sig, lam):
  p =lam* norm.pdf(val, mu, sig)
  return p

In [8]:
prob(1,2,3,4)

0.503177636923991

#  Expectation

In [9]:
## assign every data point to its most likely cluster
# Classification
def expectation(dataFrame, parameters):
  for i in range(dataFrame.shape[0]):
    x = dataFrame['x'][i]
    #y = dataFrame['y'][i]
    #print( i , '\n')
    p_cluster1 = prob(x, parameters['mu1'][0], parameters['sig1'][0], parameters['lambda1'][0])
    p_cluster1
    p_cluster2 = prob(x, parameters['mu2'][0], parameters['sig2'][0], parameters['lambda2'][0])
    if p_cluster1 > p_cluster2:
      dataFrame['label'][i] = 1
    else:
      dataFrame['label'][i] = 2
  return dataFrame

In [10]:
norm.pdf(2,2,pow(2*math.pi,-0.5))

1.0

# Maximization

In [11]:
# update estimates of lambda, mu and sigma

def maximization(dataFrame, parameters):
  points_assigned_to_cluster1 = dataFrame[dataFrame['label'] == 1]
  points_assigned_to_cluster2 = dataFrame[dataFrame['label'] == 2]
  percent_assigned_to_cluster1 = len(points_assigned_to_cluster1) / float(len(dataFrame))
  percent_assigned_to_cluster2 = 1 - percent_assigned_to_cluster1
  parameters['lambda1'] = [percent_assigned_to_cluster1]
  parameters['lambda2'] = [percent_assigned_to_cluster2]
  parameters['mu1'] = [points_assigned_to_cluster1['x'].mean()]
  parameters['mu2'] = [points_assigned_to_cluster2['x'].mean()]
  parameters['sig1'] =  [points_assigned_to_cluster1['x'].std()]
  parameters['sig2'] =  [points_assigned_to_cluster2['x'].std()] 
  return parameters

In [12]:
# get the distance between points
# used for determining if params have converged
def distance(old_params, new_params):
  dist = 0
  for param in ['mu1', 'mu2']:
    for i in range(len(old_params)):
      dist += (old_params[param][i] - new_params[param][i]) ** 2
  return dist ** 0.5

In [13]:
# loop until parameters converge
shift = maxsize
epsilon = 0.01
iters = 0
df_copy = df.copy()
# randomly assign points to their initial clusters
df_copy['label'] = map(lambda x: x+1, np.random.choice(2, len(df)))
params = pd.DataFrame(guess)

while shift > epsilon:
  iters += 1
  # E-step
  updated_labels = expectation(df_copy.copy(), params)

  # M-step
  updated_parameters = maximization(updated_labels, params.copy())

  # see if our estimates of mu have changed
  # could incorporate all params, or overall log-likelihood
  shift = distance(params, updated_parameters)

  # logging
  print("iteration {}, shift {}".format(iters, shift))

  # update labels and params for the next iteration
  df_copy = updated_labels
  params = updated_parameters
  print(params)


#   fig = plt.figure()
#   plt.scatter(df_copy['x'], df_copy['y'], 24, c=df_copy['label'])
#   fig.savefig("iteration{}.png".format(iters))

C:\Users\range\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\range\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


iteration 1, shift 2.652543167015592
        mu1      sig1       mu2      sig2  lambda1  lambda2
0 -0.021017  1.331231  7.465274  2.350548      0.4      0.6
iteration 2, shift 0.20937561621856404
        mu1      sig1       mu2      sig2  lambda1  lambda2
0  0.110546  1.428409  7.628152  2.215648     0.42     0.58
iteration 3, shift 0.10538414229072439
        mu1      sig1       mu2      sig2  lambda1  lambda2
0  0.177544  1.478103  7.709498  2.146186     0.43     0.57
iteration 4, shift 0.0
        mu1      sig1       mu2      sig2  lambda1  lambda2
0  0.177544  1.478103  7.709498  2.146186     0.43     0.57


In [14]:
params3

NameError: name 'params3' is not defined

In [ ]:
params2

In [ ]:
params1